This notebook serves as an example of how to load and manipulate the [QM9 dataset](https://figshare.com/collections/Quantum_chemistry_structures_and_properties_of_134_kilo_molecules/978904) using a `Dataset` object.

# Imports

In [1]:
import os
import numpy as np

from ase import Atoms

# Initialize the `HDF5Client`

In [2]:
from colabfit.tools.database import MongoDatabase

client = MongoDatabase('qm9', nprocs=6)#, drop=True)

# Data loading

## Define the properties and reader functions

In [3]:
client.insert_property_definition({
    'property-id': 'qm9-property',
    'property-title': 'A, B, C, mu, alpha, homo, lumo, gap, r2, zpve, U0, U, H, G, Cv',
    'property-description': 'Geometries minimal in energy, corresponding harmonic frequencies, dipole moments, polarizabilities, along with energies, enthalpies, and free energies of atomization',
    'a':     {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Rotational constant A'},
    'b':     {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Rotational constant B'},
    'c':     {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Rotational constant C'},
    'mu':    {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Dipole moment'},
    'alpha': {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Isotropic polarizability'},
    'homo':  {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Energy of Highest occupied molecular orbital (HOMO)'},
    'lumo':  {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Energy of Lowest occupied molecular orbital (LUMO)'},
    'gap':   {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Gap, difference between LUMO and HOMO'},
    'r2':    {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Electronic spatial extent'},
    'zpve':  {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Zero point vibrational energy'},
    'u0':    {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Internal energy at 0 K'},
    'u':     {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Internal energy at 298.15 K'},
    'h':     {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Enthalpy at 298.15 K'},
    'g':     {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Free energy at 298.15 K'},
    'cv':    {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Heat capacity at 298.15 K'},
    'smiles-relaxed':    {'type': 'string', 'has-unit': False, 'extent': [], 'required': True, 'description': 'SMILES for relaxed geometry'},
    'inchi-relaxed':     {'type': 'string', 'has-unit': False, 'extent': [], 'required': True, 'description': 'InChI for relaxed geometry'},
})

/home/jvita/scripts/colabfit-tools/colabfit/tools/database.py:625: UserWarning: Invalid KIM property-id; Temporarily renaming to tag:@,0000-00-00:property/qm9-property
  warnings.warn(f"Invalid KIM property-id; Temporarily renaming to {dummy_dict['property-id']}")


In [2]:
client.get_property_definition('qm9-property')['definition']

{'property-id': 'qm9-property',
 'property-title': 'A, B, C, mu, alpha, homo, lumo, gap, r2, zpve, U0, U, H, G, Cv',
 'property-description': 'Geometries minimal in energy, corresponding harmonic frequencies, dipole moments, polarizabilities, along with energies, enthalpies, and free energies of atomization',
 'a': {'type': 'float',
  'has-unit': True,
  'extent': [],
  'required': True,
  'description': 'Rotational constant A'},
 'b': {'type': 'float',
  'has-unit': True,
  'extent': [],
  'required': True,
  'description': 'Rotational constant B'},
 'c': {'type': 'float',
  'has-unit': True,
  'extent': [],
  'required': True,
  'description': 'Rotational constant C'},
 'mu': {'type': 'float',
  'has-unit': True,
  'extent': [],
  'required': True,
  'description': 'Dipole moment'},
 'alpha': {'type': 'float',
  'has-unit': True,
  'extent': [],
  'required': True,
  'description': 'Isotropic polarizability'},
 'homo': {'type': 'float',
  'has-unit': True,
  'extent': [],
  'required

In [5]:
from colabfit.tools.property_settings import PropertySettings

pso_id = client.insert_property_settings(
    PropertySettings(
        method='DFT/B3LYP/6-31G(2df,p)',
        description='QM9 property settings calculation',
        files=[],
        labels=['DFT', 'B3LYP', '6-31G(2df,p)'],
    )
)

pso_id

'-993823251101522014'

## Defining a `property_map`

In [3]:
property_map = {
    'qm9-property': {
        # Property Definition field: {'field': ASE field, 'units': ASE-readable units}
        'a':     {'field': 'A',     'units': 'GHz'},
        'b':     {'field': 'B',     'units': 'GHz'},
        'c':     {'field': 'C',     'units': 'GHz'},
        'mu':    {'field': 'mu',    'units': 'Debye'},
        'alpha': {'field': 'alpha', 'units': 'Bohr*Bohr*Bohr'},
        'homo':  {'field': 'homo',  'units': 'Hartree'},
        'lumo':  {'field': 'lumo',  'units': 'Hartree'},
        'gap':   {'field': 'gap',   'units': 'Hartree'},
        'r2':    {'field': 'r2',    'units': 'Bohr*Bohr'},
        'zpve':  {'field': 'zpve',  'units': 'Hartree'},
        'u0':    {'field': 'U0',    'units': 'Hartree'},
        'u':     {'field': 'U',     'units': 'Hartree'},
        'h':     {'field': 'H',     'units': 'Hartree'},
        'g':     {'field': 'G',     'units': 'Hartree'},
        'cv':    {'field': 'Cv',    'units': 'cal/mol/K'},
        'smiles-relaxed': {'field': 'SMILES_relaxed', 'units': None},
        'inchi-relaxed': {'field': 'SMILES_relaxed',  'units': None},
    }
}

In [7]:
def reader(file_path):
    # A function for returning a list of ASE a
    
    properties_order = [
        'tag', 'index', 'A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'U0', 'U', 'H', 'G', 'Cv'
    ]
        
#     images = []
    with open(file_path, 'r') as f:
        lines = [_.strip() for _ in f.readlines()]
        
        na = int(lines[0])
        properties = lines[1].split()
        
        symbols = []
        positions = []
        partial_charges = []
        
        for line in lines[2:2+na]:
            split = line.split()
            split = [_.replace('*^', 'e') for _ in split]  # Python-readable scientific notation
            
            # Line order: symbol, x, y, z, charge
            symbols.append(split[0])
            positions.append(split[1:4])
            partial_charges.append(split[-1])
            
        positions = np.array(positions)
        partial_charges = np.array(partial_charges, dtype=float)
                
        atoms = Atoms(symbols=symbols, positions=positions)
        
        atoms.info['mulliken_partial_charges'] = partial_charges
        
        name = os.path.splitext(os.path.split(file_path)[-1])[0]

        atoms.info['name'] = name
        
        for pname, val in zip(properties_order[2:], properties[2:]):
            atoms.info[pname] = float(val)
            
        frequencies = np.array(lines[-3].split(), dtype=float)
        atoms.info['frequencies'] = frequencies
                
        smiles = lines[-2].split()
        inchi  = lines[-1].split()
        
        atoms.info['SMILES']    = smiles[0]
        atoms.info['SMILES_relaxed'] = smiles[1]
        atoms.info['InChI']     = inchi[0]
        atoms.info['InChI_relaxed']  = inchi[1]
        
        yield atoms
#         images.append(atoms)
    
#     return images

## Load and insert the data

In [8]:
from colabfit.tools.database import load_data

images = list(load_data(
    file_path='/home/jvita/scripts/colabfit/data/quantum-machine/qm9',
    file_format='folder',
    name_field='name',  # key in Configuration.info to use as the Configuration name
    elements=['H', 'C', 'N', 'O', 'F'],    # order matters for CFG files, but not others
    default_name='qm9',  # default name with `name_field` not found
    reader=reader,
    glob_string='*.xyz',
    verbose=True
))

Loading data: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 139980/139980 [01:53<00:00, 1229.64it/s]


In [9]:
ids = list(client.insert_data(
    images,
    property_map=property_map,
    property_settings={'qm9-property': pso_id},
    generator=False,
    verbose=True
))

/home/jvita/Programs/miniconda2/envs/py38/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
Adding configurations to Database:   0%|                                                                                                      | 0/23330 [00:00<?, ?it/s]/home/jvita/scripts/colabfit-tools/colabfit/tools/property.py:231: UserWarning: Invalid KIM property-id; Temporarily renaming to tag:@,0000-00-00:property/qm9-property
  warnings.warn(f"Invalid KIM property-id; Temporarily renaming to {dummy_dict['property-id']}")
Adding configurations to Database:   4%|███▌                                                                                       | 915/23330 [00:01<00:

Writing configurations: 23330


Adding configurations to Database: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 23330/23330 [00:44<00:00, 524.45it/s]


Writing configurations: 23330


Adding configurations to Database: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 23330/23330 [00:44<00:00, 523.00it/s]


...

Writing properties: 23330


Adding configurations to Database:  97%|██████████████████████████████████████████████████████████████████████████████████████▋  | 22711/23330 [00:44<00:01, 502.22it/s]

Writing properties: 23330


Adding configurations to Database: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 23330/23330 [00:46<00:00, 505.14it/s]


Writing configurations: 23330
Writing properties: 23330
Writing properties: 23330
Writing property settings: 1
Writing property settings: 1
Writing properties: 23330
Writing property settings: 1
Writing property settings: 1
Writing properties: 23330
Writing property settings: 1
Writing property settings: 1


In [11]:
client.get_property_settings(pso_id)

PropertySettings(method='DFT/B3LYP/6-31G(2df,p)', description='QM9 property settings calculation', labels={'6-31G(2df,p)', 'DFT', 'B3LYP'})

In [20]:
list(client.configurations.find({'relationships.properties.1': {'$exists': True}}, {'relationships.properties': 1, 'names': 1}))

[{'_id': '3656036745970248267',
  'names': ['dsC7O2H10nsd_0776', 'dsgdb9nsd_047894'],
  'relationships': {'properties': ['-4308075469529548156',
    '8798883636898321313']}},
 {'_id': '-3845410279001134360',
  'names': ['dsC7O2H10nsd_0290', 'dsgdb9nsd_039272'],
  'relationships': {'properties': ['2878053599155571517',
    '-8941814523317274516']}}]

# Defining `ConfigurationSet`s

In [4]:
co_ids1 = [_['_id'] for _ in client.configurations.find({'names': {'$regex': 'dsC7O2H10nsd'}}, {'_id': 1})]
co_ids2 = [_['_id'] for _ in client.configurations.find({'names': {'$regex': 'dsgdb9nsd'}}, {'_id': 1})]

print('Database 1', '(dsC7O2H10nsd):'.rjust(15), len(co_ids1))
print('Database 2', '(dsgdb9nsd):'.rjust(15), len(co_ids2))

Database 1 (dsC7O2H10nsd): 6095
Database 2    (dsgdb9nsd): 133885


In [5]:
cs_id1 = client.insert_configuration_set(
    co_ids1,
    description='Isomers of C7O2H10'
)
cs_id1

'-2437119122457883750'

In [6]:
cs_id2 = client.insert_configuration_set(
    co_ids2,
    description='The subset of all 133,885 species with up to '\
    'nine heavy atoms (CONF) out of the GDB-17 chemical universe '\
    'of 166 billion organic molecules'
)
cs_id2

'2955061802318442107'

In [7]:
cs1 = client.get_configuration_set(cs_id1)['configuration_set']

print(cs1.description)

for k,v in cs1.aggregated_info.items():
    print(k, v)

Isomers of C7O2H10
nconfigurations 6095
nsites 115805
nelements 3
elements ['C', 'H', 'O']
individual_elements_ratios [[0.37], [0.53], [0.11]]
total_elements_ratios [0.3684210526315789, 0.5263157894736842, 0.10526315789473684]
labels []
labels_counts []
chemical_formula_reduced ['C7H10O2']
chemical_formula_anonymous ['A10B7C2']
chemical_formula_hill ['C7H10O2']
nperiodic_dimensions [0]
dimension_types [[0, 0, 0]]


In [8]:
cs2 = client.get_configuration_set(cs_id2)['configuration_set']

print(cs2.description)

for k,v in cs2.aggregated_info.items():
    print(k, v)

The subset of all 133,885 species with up to nine heavy atoms (CONF) out of the GDB-17 chemical universe of 166 billion organic molecules
nconfigurations 133885
nsites 2407753
nelements 5
elements ['C', 'H', 'N', 'O', 'F']
individual_elements_ratios [[0.33, 0.36, 0.43, 0.32, 0.35, 0.38, 0.28, 0.37, 0.3, 0.31, 0.25, 0.42, 0.5, 0.67, 0.75, 0.26, 0.18, 0.6, 0.11, 0.27, 0.69, 0.44, 0.19, 0.53, 0.7, 0.78, 0.2, 0.45, 0.62, 0.12, 0.29, 0.21, 0.15, 0.23, 0.22, 0.54, 0.46, 0.17, 0.1, 0.09, 0.71, 0.47, 0.55, 0.8, 0.39, 0.14, 0.64, 0.56, 0.4, 0.57, 0.24, 0.41, 0.58], [0.5, 0.36, 0.6, 0.53, 0.57, 0.44, 0.47, 0.55, 0.64, 0.43, 0.42, 0.59, 0.25, 0.67, 0.75, 0.18, 0.11, 0.27, 0.69, 0.61, 0.7, 0.2, 0.45, 0.62, 0.12, 0.29, 0.1, 0.54, 0.71, 0.22, 0.17, 0.46, 0.38, 0.63, 0.3, 0.8, 0.14, 0.31, 0.56, 0.73, 0.4, 0.65, 0.58, 0.33], [0.06, 0.12, 0.19, 0.14, 0.31, 0.07, 0.25, 0.42, 0.24, 0.45, 0.5, 0.67, 0.75, 0.18, 0.43, 0.6, 0.35, 0.11, 0.05, 0.27, 0.44, 0.36, 0.04, 0.28, 0.2, 0.62, 0.29, 0.21, 0.17, 0.1, 0.

# Defining `Dataset`s

In [9]:
pr_ids = [_['_id'] for _ in client.properties.find({}, {'_id': 1})]
len(pr_ids)

139980

In [10]:
ds_id = client.insert_dataset(
    cs_ids=[cs_id1, cs_id2],
    pr_ids=pr_ids,
    authors=['Raghunathan Ramakrishnan', 'Pavlo Dral', 'Matthias Rupp', 'O. Anatole von Lilienfeld'],
    links=[
        'https://www.nature.com/articles/sdata201422',
        'https://figshare.com/collections/Quantum_chemistry_structures_and_properties_of_134_kilo_molecules/978904'
    ],
    description="The QM9 dataset, split into the GDB-9 molecules and the C7O2H10 isomers",
    resync=True
)
ds_id

KeyboardInterrupt: 

In [ ]:
dataset = Dataset('dsgdb9nsd')

dataset.authors = [
    'Raghunathan Ramakrishnan', 'Pavlo Dral', 'Matthias Rupp', 'O. Anatole von Lilienfeld'
]

dataset.links = [
    'https://www.nature.com/articles/sdata201422',
    'https://figshare.com/collections/Quantum_chemistry_structures_and_properties_of_134_kilo_molecules/978904'
]

dataset.description = \
"These molecules correspond to the subset of all 133,885 "\
"species with up to nine heavy atoms (CONF) out of the GDB-17 "\
"chemical universe of 166 billion organic molecules."

In [ ]:
dataset.configurations = load_data(
    file_path='qm9',
    file_format='folder',
    name_field='name',  # key in Configuration.info to use as the Configuration name
    elements=['H', 'C', 'N', 'O', 'F'],    # order matters for CFG files, but not others
    default_name='qm9',  # default name with `name_field` not found
    reader=reader,
    glob_string='*.xyz',
    verbose=True
)

In [ ]:
dataset.property_map = property_map

dataset.custom_definitions = {'qm9-property': qm9_property_definition}

In [ ]:
dataset.parse_data(convert_units=False, verbose=True)

In [ ]:
dataset.property_settings_regexes = {
    '.*':
        PropertySettings(
            method='DFT/B3LYP/6-31G(2df,p)',
            description='QM9 property settings calculation',
            files=[],
            labels=['DFT', 'B3LYP', '6-31G(2df,p)'],
        )
}

In [ ]:
dataset.resync()

In [ ]:
dataset.to_markdown(
    base_folder='/content',
    html_file_name='README.md',
    data_file_name=dataset.name+'.extxyz',
    data_format='xyz',
)

# Exploration

Results of running command:

`dataset.plot_histograms(['a', 'b', 'c', 'mu', 'alpha', 'homo', 'lumo', 'r2', 'zpve', 'u0', 'u', 'h', 'g', 'cv'])`

[QM9 histograms (original)](https://github.com/colabfit/colabfit-tools/blob/master/colabfit/examples/qm9_histograms.png)

In [ ]:
print(dataset.get_statistics('a'))
print(dataset.get_statistics('b'))
print(dataset.get_statistics('c'))

In [ ]:
clean = dataset.filter(
    'data',
    lambda p: (p['a']['source-value'] < 20) and (p['b']['source-value'] < 10),
    verbose=True
)

In [ ]:
clean.plot_histograms(['a', 'b', 'c', 'mu', 'alpha', 'homo', 'lumo', 'r2', 'zpve', 'u0', 'u', 'h', 'g', 'cv'])

Results of running command:

`clean.plot_histograms(['a', 'b', 'c', 'mu', 'alpha', 'homo', 'lumo', 'r2', 'zpve', 'u0', 'u', 'h', 'g', 'cv'])`

[QM9 histograms (clean)](https://github.com/colabfit/colabfit-tools/blob/master/colabfit/examples/qm9_clean_histograms.png)

In [ ]:
print(clean.get_statistics('a'))
print(clean.get_statistics('b'))
print(clean.get_statistics('c'))